In [2]:
import mysql.connector
import json, os, re
from datetime import datetime

with open("id_resolver_map.json", "r", encoding="utf-8") as f:
    id_maps = json.load(f)

def resolve_id(name_string, id_type="amc_id_map"):
    regex_map = id_maps.get(id_type, {})
    for pattern, id_val in regex_map.items():
        if re.search(pattern, name_string, re.IGNORECASE):
            return id_val
    return None

def null_if_empty(value):
    if isinstance(value, str) and value.strip() == "":
        return None
    return value

def get_json_paths(root_folder):
    return [
        os.path.join(root_folder, file)
        for file in os.listdir(root_folder)
        if file.endswith(".json")
    ]

def load_all_records_mydata(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                sample_scheme = next(iter(data.values()))
                amc_name = sample_scheme.get("amc_name", "").strip()
                name = " ".join(amc_name.split(" ")[:2])
                if name not in all_records:
                    all_records[name] = data
                else:
                    name += " 2"
                    all_records[name] = data
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records


In [6]:
conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

# conn = mysql.connector.connect(
#     host="localhost",   
#     user="root",            
#     password="1234", 
#     database="cog_updated_db",  
#     port=3306,               
#     charset="utf8mb4"
# )
cursor = conn.cursor()

def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    amc_id = resolve_id(details.get("amc_name", ""), "amc_id_map")
    fund_id = resolve_id(details.get("mutual_fund_name", ""), "fund_id_map")

    keys = [
        "amc_id", "fund_id", "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name",
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]

    metrics = details.get("metrics", {})
    load = details.get("load", {})

    values = [
        amc_id,
        fund_id,
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(details.get("benchmark_index", "")),
        null_if_empty("20250228|February 2025"),
        null_if_empty(details.get("monthly_aaum_value", "")),
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(load.get("entry_load", "")),
        null_if_empty(load.get("exit_load", "")),
    ] + [null_if_empty(metrics.get(k, "")) for k in keys[18:]]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid, amc_id, fund_id
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None, None, None

def insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, amc_id, fund_id, entered_time, amc_for_month, data_from,
             amc_name, fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            amc_id,
            fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

base_dir = os.path.join(os.path.dirname(os.getcwd()), "sql_learn", "json", "FEB25DATA")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records_mydata(json_paths)
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB25"
data_from = "mydata"

for amc_name, schemes in combined_records.items():
    print(f"Doing: {amc_name}")
    for scheme_name, details in schemes.items():
        try:
            mutual_fund_id, amc_id, fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped Data Insertion for Scheme {scheme_name}")
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")
    print(f"Done {amc_name}")

conn.commit()
cursor.close()
conn.close()
print("Working!!")


Doing: 360 ONE
Done 360 ONE
Doing: Aditya Birla
Done Aditya Birla
Doing: Axis Asset
Done Axis Asset
Doing: Bajaj Finserv
Done Bajaj Finserv
Doing: Bandhan Asset
Done Bandhan Asset
Doing: Bank of
Done Bank of
Doing: Baroda BNP
Done Baroda BNP
Doing: Canara Robeco
Done Canara Robeco
Doing: DSP Investment
Done DSP Investment
Doing: Edelweiss Asset
Done Edelweiss Asset
Doing: Franklin Templeton
Done Franklin Templeton
Doing: Groww Asset
Done Groww Asset
Doing: HDFC Asset
Done HDFC Asset
Doing: HDFC Asset 2
Done HDFC Asset 2
Doing: Helios Capital
Done Helios Capital
Doing: HSBC Asset
Done HSBC Asset
Doing: ICICI Prudential
Done ICICI Prudential
Doing: ICICI Prudential 2
Done ICICI Prudential 2
Doing: Invesco Asset
Done Invesco Asset
Doing: ITI Asset
Done ITI Asset
Doing: JM Financial
Done JM Financial
Doing: Kotak Mahindra
Done Kotak Mahindra
Doing: PGIM India
Done PGIM India
Doing: LIC Mutual
Done LIC Mutual
Doing: Mahindra Manulife
Done Mahindra Manulife
Doing: Mirae Asset
Done Mirae Asse

In [1]:
import mysql.connector
import json, os, re
from datetime import datetime
import pandas as pd
import re
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\Feb-25")
from query import *

def get_json_paths(root_folder):
    json_paths = []
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isfile(folder_path) and folder_path.endswith(".json"):
            json_paths.append(folder_path)
    return json_paths

def load_all_records(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                records = data.get("records", [])
                name = " ".join(records[0].get("value", {}).get("amc_name", "").split(" ")[:2])
                if name not in all_records:
                    all_records[name] = records
                else:
                    name += " 2"
                    all_records[name] = records
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

def load_all_records_mydata(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)  # The whole dict is the data
                sample_scheme = next(iter(data.values()))
                amc_name = sample_scheme.get("amc_name", "").strip()
                name = " ".join(amc_name.split(" ")[:2])
                
                if name not in all_records:
                    all_records[name] = data
                else:
                    name += " 2"
                    all_records[name] = data
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records


with open("id_resolver_map.json", "r", encoding="utf-8") as f:
    id_maps = json.load(f)

In [ ]:
#mydatainsert
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\json\\FEB25DATA") #FEB25DATA
json_paths = get_json_paths(base_dir)
combined_records = load_all_records_mydata(json_paths)

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)
cursor = conn.cursor()

def null_if_empty(value):
    if isinstance(value, str) and value.strip() == "":
        return None
    return value


def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    keys = [
        "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name", #mutual_fund_name
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]
    
    metrics = details.get("metrics", {})
    load = details.get("load", {})
    
    values = [
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(details.get("benchmark_index", "")),
        null_if_empty("20250228|February 2025"),
        null_if_empty(details.get("monthly_aaum_value", "")),  # Or make this dynamic again
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(load.get("entry_load", "")),
        null_if_empty(load.get("exit_load", "")),
    ] + [null_if_empty(metrics.get(k, "")) for k in keys[16:]]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None

def insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, entered_time, amc_for_month, data_from, amc_name,
             mutual_fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

# Usage
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB25"
data_from = "mydata"

for amc_name, schemes in combined_records.items():
    print(f"Doing: {amc_name}")
    for scheme_name, details in schemes.items():
        try:
            mutual_fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped Data Insertion for Scheme {scheme_name}")
        except Exception as e:
            print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")
    print(f"Done {amc_name}")
conn.commit()
cursor.close()
conn.close()
print("Working!!")


In [ ]:
#finkinsert
base_dir = os.path.join(os.path.dirname(os.getcwd())+"\\sql_learn\\json\\Feb-25")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

conn = mysql.connector.connect(
    host="localhost",   
    user="root",            
    password="1234", 
    database="cog_updated_db",  
    port=3306,               
    charset="utf8mb4"
)
cursor = conn.cursor()

def null_if_empty(value):
    if isinstance(value, str) and value.strip() == "":
        return None
    return value

def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    keys = [
        "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name",
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]

    metrics = details.get("metrics", {})
    load = details.get("load", {})

    values = [
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(", ".join(details.get("benchmark_index", [])) if isinstance(details.get("benchmark_index"), list) else details.get("benchmark_index", "")),
        null_if_empty(details.get("monthly_aaum_date", "")),
        null_if_empty(details.get("monthly_aaum_value", "")),
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(next((l.get("comment") for l in details.get("load", []) if l.get("type") == "entry"), "")),
        null_if_empty(next((l.get("comment") for l in details.get("load", []) if l.get("type") == "exit"), ""))
    ] + [null_if_empty(next((m.get("value") for m in details.get("metrics", []) if m.get("name") == key), "")) for key in keys[16:]]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None

def insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, entered_time, amc_for_month, data_from, amc_name,
             mutual_fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

# Start Insertion
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB25"
data_from = "finkstein"

for amc_name, records in combined_records.items():
    print(f"Inserting AMC: {amc_name}")
    for record in records:
        try:
            details = record["value"] 
            mutual_fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped: {details.get('main_scheme_name')}")
        except Exception as e:
            print(f"Error processing scheme under AMC '{amc_name}': {e}")


conn.commit()
cursor.close()
conn.close()
print("Finkstein data insertion completed.")


In [6]:
#fink
# conn = mysql.connector.connect(
#     host="172.22.225.155",
#     user="cog_mf",
#     password="bnYwFChjLAV2Z%9E",
#     database="cog_mf",
#     port=3306,
#     charset='utf8mb4'
# )

conn = mysql.connector.connect(
    host="localhost",   
    user="root",            
    password="1234", 
    database="cog_updated_db",  
    port=3306,               
    charset="utf8mb4"
)
cursor = conn.cursor()

def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    amc_id = resolve_id(details.get("amc_name", ""), "amc_id_map")
    fund_id = resolve_id(details.get("mutual_fund_name", ""), "fund_id_map")

    keys = [
        "amc_id", "fund_id", "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name",
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]

    values = [
        amc_id,
        fund_id,
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(", ".join(details.get("benchmark_index", [])) if isinstance(details.get("benchmark_index"), list) else details.get("benchmark_index", "")),
        null_if_empty(details.get("monthly_aaum_date", "")),
        null_if_empty(details.get("monthly_aaum_value", "")),
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(next((l.get("comment") for l in details.get("load", []) if l.get("type") == "entry"), "")),
        null_if_empty(next((l.get("comment") for l in details.get("load", []) if l.get("type") == "exit"), ""))
    ] + [
        null_if_empty(next((m.get("value") for m in details.get("metrics", []) if m.get("name") == key), ""))
        for key in keys[18:]
    ]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid, amc_id, fund_id
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None, None, None

def insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, amc_id, fund_id, entered_time, amc_for_month, data_from,
             amc_name, fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            amc_id,
            fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

# Main execution
base_dir = os.path.join(os.path.dirname(os.getcwd()), "sql_learn", "json", "Feb-25")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "FEB25"
data_from = "finkstein"

for amc_name, records in combined_records.items():
    print(f"Inserting AMC: {amc_name}")
    for record in records:
        try:
            details = record["value"]
            mutual_fund_id, amc_id, fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped: {details.get('main_scheme_name')}")
        except Exception as e:
            print(f"Error processing scheme under AMC '{amc_name}': {e}")

conn.commit()
cursor.close()
conn.close()
print("Finkstein data insertion completed.")

Inserting AMC: Quant Money
Inserting AMC: Franklin Templeton
Inserting AMC: HDFC Asset
Inserting AMC: HSBC Asset
Inserting AMC: ICICI Prudential
Inserting AMC: Bandhan Asset
Inserting AMC: 360 ONE
Inserting AMC: Axis Asset
Inserting AMC: Groww Asset
Inserting AMC: Invesco Asset
Inserting AMC: JM Financial
Inserting AMC: Kotak Mahindra
Inserting AMC: LIC Mutual
Inserting AMC: Mahindra Manulife
Inserting AMC: Mirae Asset
Inserting AMC: Motilal Oswal
Inserting AMC: Baroda BNP
Inserting AMC: PPFAS Asset
Inserting AMC: Quantum Asset
Inserting AMC: Nippon Life
Inserting AMC: SBI Funds
Inserting AMC: Shriram Asset
Inserting AMC: Sundaram Asset
Inserting AMC: Tata Asset
Inserting AMC: Taurus Asset
Inserting AMC: Aditya Birla
Inserting AMC: Union Asset
Inserting AMC: UTI Asset
Inserting AMC: WhiteOak Capital
Inserting AMC: ITI Asset
Inserting AMC: Trust Asset
Inserting AMC: Navi Asset
Inserting AMC: NJ Asset
Inserting AMC: Samco Asset
Inserting AMC: Bajaj Finserv
Inserting AMC: BOI STAR
Inserti

In [ ]:
# CREATE TABLE mf_common_fund_data (
#     id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY,
#     amc_id INT(11),
#     fund_id INT(11),
#     entered_time VARCHAR(40),
#     amc_for_month VARCHAR(40),
#     data_from VARCHAR(100),
#     amc_name MEDIUMTEXT,
#     main_scheme_name MEDIUMTEXT,
#     fund_name MEDIUMTEXT,
#     benchmark_index MEDIUMTEXT,
#     monthly_aaum_date VARCHAR(50),
#     monthly_aaum_value MEDIUMTEXT,
#     scheme_launch_date MEDIUMTEXT,
#     min_addl_amt VARCHAR(150),
#     min_addl_amt_multiple VARCHAR(150),
#     min_amt VARCHAR(150),
#     min_amt_multiple VARCHAR(150),
#     entry_load MEDIUMTEXT,
#     exit_load MEDIUMTEXT,
#     alpha VARCHAR(140),
#     arithmetic_mean_ratio VARCHAR(140),
#     average_div_yld VARCHAR(140),
#     average_pb VARCHAR(140),
#     average_pe VARCHAR(140),
#     avg_maturity VARCHAR(140),
#     beta VARCHAR(140),
#     correlation_ratio VARCHAR(140),
#     downside_deviation VARCHAR(140),
#     information_ratio VARCHAR(140),
#     macaulay VARCHAR(140),
#     mod_duration VARCHAR(140),
#     port_turnover_ratio VARCHAR(140),
#     r_squared_ratio VARCHAR(140),
#     roe_ratio VARCHAR(140),
#     sharpe VARCHAR(140),
#     sortino_ratio VARCHAR(140),
#     std_dev VARCHAR(140),
#     tracking_error VARCHAR(140),
#     treynor_ratio VARCHAR(140),
#     upside_deviation VARCHAR(140),
#     ytm VARCHAR(140)
# );

# CREATE TABLE mf_common_fund_manager_data (
#     id INT(11) NOT NULL AUTO_INCREMENT PRIMARY KEY,
#     amc_id INT(11),
#     fund_id INT(11),
#     mutual_fund_id INT(11),
#     entered_time VARCHAR(40),
#     amc_for_month VARCHAR(40),
#     data_from VARCHAR(100),
#     amc_name TEXT,
#     fund_name TEXT,
#     main_scheme_name TEXT,
#     name TEXT,
#     qualification TEXT,
#     managing_fund_since TEXT,
#     total_exp TEXT
# );